# AR status

In [45]:
import glob
import numpy as np
import xarray as xr
import os

# READ input data

In [2]:
lh_files = glob.glob('/home/hanna/lagrings/ERA5_monthly/*2014*01*.nc')

In [3]:
lh_files

['/home/hanna/lagrings/ERA5_monthly/2014_01_q.nc',
 '/home/hanna/lagrings/ERA5_monthly/2014_01_r.nc',
 '/home/hanna/lagrings/ERA5_monthly/2014_01_t2m.nc',
 '/home/hanna/lagrings/ERA5_monthly/2014_01_sp.nc',
 '/home/hanna/lagrings/ERA5_monthly/2014_01_tcc.nc']

In [40]:
longitude  = np.arange(-15.0, 25.0+0.25, step = 0.25)
latitude   =  np.arange(30.0, 50.0+0.25, step = 0.25)

In [17]:
#input_data

In [6]:
lat = 30.0
lon = 0.0

In [17]:
inp = input_data.sel(latitude=lat, longitude=lon)

In [20]:
sub = inp.isel(time=0)

# Load weights

In [2]:
glob.glob('/home/hanna/EX3_Results_AR_STR/*')

['/home/hanna/EX3_Results_AR_STR/AR-B-5',
 '/home/hanna/EX3_Results_AR_STR/AR-0']

In [12]:
files = glob.glob('/home/hanna/EX3_Results_AR_STR/AR-B-5/*weights*L1*_{}_*{}.nc*'.format(lon, lat))

In [20]:
files[0]

'/home/hanna/EX3_Results_AR_STR/AR-B-5/weights_AR-B-5-L1_5.25_45.75.nc'

In [14]:
data = xr.open_dataset(files[0])

In [23]:
variables = ['q', 't2m', 'r', 'sp']

In [35]:
data.coeffs.values

array([ 6.39408076e+00, -1.31813609e-03, -3.08693608e-04, -1.68213026e-05,
        2.02346748e+00,  8.31700013e-01])

In [27]:
float(sub.q.values)

0.005230146460235119

In [28]:
float(sub.r.values)

89.20773315429688

In [33]:
prev_tcc = float(sub.tcc.values)
arr = np.array([float(sub.q.values), float(sub.t2m.values), float(sub.r.values), float(sub.sp.values), 1, prev_tcc])

In [36]:
prev_tcc = arr.T@data.coeffs.values

0.032068967034164064

In [ ]:
#data = xr.open_mfdataset(files, combine='by_coords')
data['latitude'] = data.latitude.values.astype(float)
data['longitude'] = data.longitude.values.astype(float)
#data = data.sortby('longitude')

In [ ]:
data

# Need to loop over to make the prediction since the prediction becomes the input in the next step

In [46]:
prediction = np.nan*np.ones((24, 81, 161)) #  # (sequence, latitude, longitude)
input_data = xr.open_mfdataset(lh_files, combine='by_coords').sel(time='2014-01-01')
longitude  = np.arange(-15.0, 25.0+0.25, step = 0.25)
latitude   =  np.arange(30.0, 50.0+0.25, step = 0.25)

for j, lat in enumerate(latitude):
    for k, lon in enumerate(longitude):
        
        files = glob.glob('/home/hanna/EX3_Results_AR_STR/AR-B-5/*weights*L1*_{}_*{}.nc*'.format(lon, lat))
        data = xr.open_dataset(files[0])
        coefs = data.coeffs.values
        inp = input_data.sel(latitude=lat, longitude=lon)
        
        for i in range(24):
            
            sub = inp.isel(time=i)
            if i==0:
                # bruk første til å lage prediksjonen. Burde jeg brukt nyyåtsaften til å sette i gang prediksjonen..?
                prev_tcc = float(sub.tcc.values)

            arr = np.array([float(sub.q.values), float(sub.t2m.values), float(sub.r.values), float(sub.sp.values), 1, prev_tcc])  
            prev_tcc = arr.T@coefs     
            prediction[i, j, k] = prev_tcc

    data_dict = {'tcc': (['sequence_length', 'latitude', 'longitude'], prediction)}

    ds = xr.Dataset(data_dict,
             coords={'longitude': (['longitude'], longitude),
                     'latitude': (['latitude'], latitude),
                     'sequence_length': (['sequence_length'], np.arange(24))
                    })
    ds['date_seq']  = '2014-01-01'
    print(ds)
    ds.to_netcdf(os.path.join('/home/hanna/MS-thesis/python_figs/','prediction.nc'))


<xarray.Dataset>
Dimensions:          (latitude: 81, longitude: 161, sequence_length: 24)
Coordinates:
  * longitude        (longitude) float64 -15.0 -14.75 -14.5 ... 24.5 24.75 25.0
  * latitude         (latitude) float64 30.0 30.25 30.5 ... 49.5 49.75 50.0
  * sequence_length  (sequence_length) int64 0 1 2 3 4 5 6 ... 18 19 20 21 22 23
Data variables:
    tcc              (sequence_length, latitude, longitude) float64 0.1632 ... nan
    date_seq         <U10 '2014-01-01'
<xarray.Dataset>
Dimensions:          (latitude: 81, longitude: 161, sequence_length: 24)
Coordinates:
  * longitude        (longitude) float64 -15.0 -14.75 -14.5 ... 24.5 24.75 25.0
  * latitude         (latitude) float64 30.0 30.25 30.5 ... 49.5 49.75 50.0
  * sequence_length  (sequence_length) int64 0 1 2 3 4 5 6 ... 18 19 20 21 22 23
Data variables:
    tcc              (sequence_length, latitude, longitude) float64 0.1632 ... nan
    date_seq         <U10 '2014-01-01'
<xarray.Dataset>
Dimensions:          (latit

IndexError: list index out of range

# How to save a file 

In [42]:
import os

In [44]:
ds.to_netcdf(os.path.join('/home/hanna/MS-thesis/python_figs/','prediction.nc'))

In [ ]:
data_dict = {'tcc': (['sequence_length', 'latitude', 'longitude'], prediction)}

longitude  = np.arange(-15.0, 25.0+0.25, step = 0.25)
latitude   =  np.arange(30.0, 50.0+0.25, step = 0.25)
#seq_length = sequence_prediction
#time = np.arange(seq_length)
#print(sequence_prediction[:, :, :, :, 0].shape)
ds = xr.Dataset(data_dict,
             coords={'longitude': (['longitude'], longitude),
                     'latitude': (['latitude'], latitude),
                     'sequence_length': (['sequence_length'], np.arange(24))
                    })
ds['date_seq']  = '2014-01-01'
print(ds)
ds.to_netcdf(os.path.join('/home/hanna/MS-thesis/python_figs/','prediction.nc'), engine = 'h5netcdf')


# BEST AR MODEL 
* Regenerating this file.

In [12]:
filesw = glob.glob('/home/hanna/EX3_Results_AR_STR/AR-B-5/*weight*L1*')

In [13]:
len(filesw)

13041

In [15]:
dataw = xr.open_dataset(filesw[0])

In [20]:
dataw.weights.values

array(['q', 't2m', 'r', 'sp'], dtype=object)

In [21]:
dataw.coeffs.values

array([1., 1., 1., 1.])

In [22]:
# forferdelig rar modell.....

In [14]:
dataw = xr.open_mfdataset(filesw, combine='by_coords')

ValueError: Coordinate variable weights is neither monotonically increasing nor monotonically decreasing on all datasets

In [ ]:
dataw